In [1]:
import h5py
import pandas as pd
import numpy as np

In [2]:
file_path = "FS-MSSM7-random.hdf5"

In [12]:
with h5py.File(file_path, 'r') as file:
    print("root name: ", file.name)
    print("Members:")
    for member in file:
        print(member, f"type {type(member)}")
    member_list = list(file.keys())
    dataset_list = []
    print("member list", member_list)
    for member in member_list:
        if isinstance(file[member], h5py.Group):
            print(member, f" is a {type(file[member])}")
            for item in file[member].keys():
                if isinstance(file[member][item], h5py.Dataset):
                    dataset_list.append(file[member][item])
                else:
                    raise TypeError("chaim is not flat something is wrong with the input file")
        elif isinstance(file[member], h5py.Dataset):
            print(member, f" is a {type(file[member])}")
            dataset_list.append(file[member])

    print("Datasets: ", len(dataset_list))
    for dset in dataset_list:
        print("Shape ", dset.shape, " DType ", dset.dtype)
    print("Dataset contents: ")
    print(dataset_list[0])
    print(type(dataset_list[0][:]))



root name:  /
Members:
MSSM type <class 'str'>
member list ['MSSM']
MSSM  is a <class 'h5py._hl.group.Group'>
Datasets:  852
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  (10000,)  DType  float64
Shape  (10000,)  DType  int32
Shape  

In [55]:
def find_groups(file_path: str, group: str | None = None, group_list: list[str] | None = None) -> list[str]:
    """
    recursively searching the file for all groups.

    parameters:
        file_path:  path to the hdf5 file
        group:      (optional) start the recursive search from this group. If not provided the search will start from the root group '/'
        group_list: (optional) All found groups will be appended to this list.

    returns:
        list[str]:  List containing all found groups as paths. Therefore subgroups also contain information about their location in the file structure
    """
    group_path: str = "/" if group is None else group
    group_path = group_path if group_path.startswith("/") else "/" + group_path
    group_path = group_path if group_path.endswith("/") else group_path + "/"

    group_list: list[str] = [] if group_list is None else group_list
    if group_path not in group_list: group_list.append(group_path)
    file = h5py.File(file_path, 'r')
    
    if isinstance(file[group_path], h5py.Group):
        initial_group: h5py.Group = file[group_path]
    else:
        raise TypeError(f"file[group] is not of type 'h5py.Group' but is '{type(file[group_path])}'")
    
    for key in initial_group.keys():
        if isinstance(initial_group[key], h5py.Group):
            new_group_path: str = group_path + key + "/"
            group_list.append(new_group_path)
            find_groups(file_path, group=new_group_path, group_list=group_list)

    file.close()

    return group_list


In [70]:
def find_datasets(file_path: str, group: str | None = None) -> list[str]:
    """
    recursivly searching the file for all datasets

    parameters:
        file_path:      path to the hdf5 file
        group:          (optional) start the recursive search from this group. If not provided the search will start from the root group '/'

    returns:
        list[str]:      List containing all found datasets as paths so the file structure can be accessed directly
    """
    group_path: str = "/" if group is None else group
    group_list: list[str] = find_groups(file_path=file_path, group=group_path)
    file = h5py.File(file_path, 'r')
    dataset_list = []

    for group in group_list:
        if isinstance(file[group], h5py.Group):
            group_obj: h5py.Group = file[group]
        else:
            raise TypeError(f"'find_groups()' found an object of type '{type(file[group])}'. Please contact support")

        for dset in group_obj.keys():
            if isinstance(group_obj[dset], h5py.Dataset):
                new_dataset_path: str = group + dset
                dataset_list.append(new_dataset_path)
            else:
                break
    
    file.close()

    return dataset_list 

In [71]:
groups = find_groups(file_path)
for group in groups:
    print(group)

/
/MSSM/


In [72]:
datasets = find_datasets(file_path)
for dset in datasets:
    print(dset)
print(len(datasets))

/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::Ad_3
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::Ad_3_isvalid
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::Au_3
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::Au_3_isvalid
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::M2
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::M2_isvalid
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::Qin
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::Qin_isvalid
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::SignMu
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::SignMu_isvalid
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::TanBeta
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::TanBeta_isvalid
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::mHd2
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::mHd2_isvalid
/MSSM/#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::

In [73]:
file = h5py.File(file_path, 'r')
file[datasets[0]]

<HDF5 dataset "#MSSM7atQ_parameters @MSSM7atQ::primary_parameters::Ad_3": shape (10000,), type "<f8">